# 🌱 CloudGarden - Smart Plant Disease Detection System## Refactored Version**Changes from original:**- All imports consolidated- Configuration centralized  - Code duplication removed- Better error handling- Cleaner organization

## 📦 1. Dependencies

In [ ]:
# Install all required packages
!pip install -q --upgrade gradio pandas matplotlib python-docx plotly
!pip install -q --upgrade firebase-admin gdown transformers torch
!pip install -q --upgrade beautifulsoup4 lxml nltk requests numpy


## 📚 2. Imports (Consolidated)

In [ ]:
# =============================================================================
# ALL IMPORTS - SINGLE LOCATION
# =============================================================================

# Standard library
import os
import re
import json
import time
import tempfile
import warnings
from datetime import datetime, timedelta, timezone
from collections import defaultdict
from typing import Dict, List, Optional, Tuple, Any
from urllib.parse import quote

# Data processing
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Web & HTTP
import requests
from bs4 import BeautifulSoup

# Firebase
import firebase_admin
from firebase_admin import credentials, db
import gdown

# NLP
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# ML
from transformers import pipeline

# Document generation
from docx import Document
from docx.enum.text import WD_ALIGN_PARAGRAPH
from zoneinfo import ZoneInfo

# UI
import gradio as gr

# Initialize NLP tools
nltk.download("stopwords", quiet=True)
nltk.download("punkt", quiet=True)
warnings.filterwarnings('ignore')

print("✅ All imports loaded successfully")


## ⚙️ 3. Configuration (Centralized)

In [ ]:
# =============================================================================
# ALL CONFIGURATION - SINGLE LOCATION
# =============================================================================

# -----------------------------------------------------------------------------
# Firebase Configuration
# -----------------------------------------------------------------------------
FIREBASE_KEY_ID = '1ESnh8BIbGKrVEijA9nKNgNJNdD5kAaYC'
FIREBASE_KEY_FILE = 'firebase_key.json'
FIREBASE_URL = "https://cloud-81451-default-rtdb.europe-west1.firebasedatabase.app/"
FIREBASE_DB_URL = FIREBASE_URL  # Alias for clarity

# -----------------------------------------------------------------------------
# External Server Configuration
# -----------------------------------------------------------------------------
BASE_URL = "https://server-cloud-v645.onrender.com/"
DEFAULT_FEED = "json"
BATCH_LIMIT = 200
REQUEST_TIMEOUT = 30

# -----------------------------------------------------------------------------
# Sensor Configuration (NO MORE MAGIC NUMBERS!)
# -----------------------------------------------------------------------------
SENSOR_VALIDATION_RANGES = {
    'temperature': {'min': -50, 'max': 100, 'unit': '°C'},
    'humidity': {'min': 0, 'max': 100, 'unit': '%'},
    'soil': {'min': 0, 'max': 100, 'unit': '%'}
}

PLANT_OPTIMAL_RANGES = {
    'temperature': {'min': 18, 'max': 32, 'margin': 1, 'unit': '°C'},
    'humidity': {'min': 35, 'max': 75, 'margin': 3, 'unit': '%'},
    'soil': {'min': 20, 'max': 60, 'margin': 3, 'unit': '%'}
}

# -----------------------------------------------------------------------------
# ML Model Configuration
# -----------------------------------------------------------------------------
PLANT_DISEASE_MODEL = "linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification"
RAG_MODEL_PRIMARY = "google/flan-t5-small"
RAG_MODEL_FALLBACK = "google/flan-t5-base"
RAG_MAX_NEW_TOKENS = 160
RAG_SNIPPET_CHARS = 300

# -----------------------------------------------------------------------------
# UI Configuration
# -----------------------------------------------------------------------------
APP_TITLE = "🌱 CloudGarden"
APP_SUBTITLE = "Smart Plant Disease Detection System"

# Colors
COLORS = {
    'temperature': '#ef4444',  # Red
    'humidity': '#3b82f6',     # Blue
    'soil': '#8b5cf6',         # Purple
    'status_ok': '#2ca02c',    # Green
    'status_warn': '#ffbf00',  # Yellow
    'status_bad': '#d62728'    # Red
}

# For backward compatibility
COLOR_TEMP = COLORS['temperature']
COLOR_HUM = COLORS['humidity']
COLOR_SOIL = COLORS['soil']
STATUS_OK_COLOR = COLORS['status_ok']
STATUS_WARN_COLOR = COLORS['status_warn']
STATUS_BAD_COLOR = COLORS['status_bad']

# -----------------------------------------------------------------------------
# Document URLs for RAG
# -----------------------------------------------------------------------------
DOC_URLS = [
    "https://doi.org/10.1038/s41598-025-20629-y",
    "https://doi.org/10.3389/fpls.2016.01419",
    "https://doi.org/10.1038/s41598-025-05102-0",
    "https://doi.org/10.1038/s41598-025-04758-y",
    "https://doi.org/10.2174/0118743315321139240627092707",
]

# -----------------------------------------------------------------------------
# HTTP Configuration
# -----------------------------------------------------------------------------
BROWSER_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

print("✅ Configuration loaded")
print(f"   Firebase URL: {FIREBASE_URL[:50]}...")
print(f"   Server URL: {BASE_URL}")
print(f"   Batch Limit: {BATCH_LIMIT}")


## 🛠️ 4. Utility Functions

In [ ]:
# =============================================================================
# UTILITY FUNCTIONS - Reusable helpers
# =============================================================================

def validate_sensor_value(value: float, sensor_type: str) -> float:
    """Validate and clip sensor value to valid range."""
    ranges = SENSOR_VALIDATION_RANGES.get(sensor_type, {'min': 0, 'max': 100})
    return max(ranges['min'], min(ranges['max'], float(value)))


def check_optimal_range(value: float, sensor_type: str) -> Tuple[bool, bool, str]:
    """
    Check if value is in optimal range.
    Returns: (is_ok, is_warning, message)
    """
    config = PLANT_OPTIMAL_RANGES.get(sensor_type)
    if not config:
        return True, False, "Unknown sensor type"
    
    min_val, max_val = config['min'], config['max']
    margin = config.get('margin', 0)
    unit = config.get('unit', '')
    
    if not (min_val <= value <= max_val):
        return False, False, f"{sensor_type.title()} out of range ({value:.1f}{unit})"
    elif value <= min_val + margin or value >= max_val - margin:
        return True, True, f"{sensor_type.title()} near limit ({value:.1f}{unit})"
    else:
        return True, False, f"{sensor_type.title()} OK ({value:.1f}{unit})"


def parse_timestamp(ts_value, source: str = "unknown") -> Optional[pd.Timestamp]:
    """Universal timestamp parser - handles all formats."""
    if ts_value is None:
        return None
    
    try:
        # Already a timestamp
        if isinstance(ts_value, pd.Timestamp):
            return ts_value
        
        # String timestamp
        if isinstance(ts_value, str):
            return pd.to_datetime(ts_value, errors='coerce', utc=True)
        
        # Numeric timestamp (unix)
        if isinstance(ts_value, (int, float)):
            # Detect milliseconds vs seconds
            if ts_value > 1e12:  # Milliseconds
                return pd.to_datetime(ts_value, unit='ms', utc=True)
            else:  # Seconds
                return pd.to_datetime(ts_value, unit='s', utc=True)
        
        return pd.to_datetime(ts_value, errors='coerce', utc=True)
    except Exception as e:
        print(f"Warning: Failed to parse timestamp from {source}: {e}")
        return None


def safe_float(value, default: float = 0.0) -> float:
    """Safely convert to float."""
    try:
        return float(value)
    except (ValueError, TypeError):
        return default


def format_timestamp_key(ts_string: str) -> str:
    """Convert timestamp to Firebase-safe key."""
    return ts_string.replace(':', '-').replace('.', '-')


def normalize_series(series: pd.Series) -> pd.Series:
    """Normalize a series to 0-1 range."""
    mn, mx = float(series.min()), float(series.max())
    if mx - mn == 0:
        return series * 0.0
    return (series - mn) / (mx - mn)


print("✅ Utility functions loaded")


## 🔥 5. Firebase Service

In [ ]:
# =============================================================================
# FIREBASE SERVICE - All Firebase operations in one place
# =============================================================================

class FirebaseService:
    """Centralized Firebase operations."""
    
    _initialized = False
    _http_session = requests.Session()
    
    @classmethod
    def initialize(cls) -> bool:
        """Initialize Firebase connection."""
        if cls._initialized:
            return True
        
        # Download credentials if needed
        if os.path.exists(FIREBASE_KEY_FILE):
            os.remove(FIREBASE_KEY_FILE)
        
        print('📥 Downloading Firebase credentials...')
        try:
            url = f'https://drive.google.com/uc?id={FIREBASE_KEY_ID}'
            gdown.download(url, FIREBASE_KEY_FILE, quiet=True, fuzzy=True)
            
            with open(FIREBASE_KEY_FILE, 'r') as f:
                creds = json.load(f)
            print(f'✓ Project: {creds.get("project_id")}')
        except Exception as e:
            print(f'⚠️ Auto-download failed: {e}')
            print('Please upload firebase_key.json manually')
            return False
        
        # Initialize Firebase Admin SDK
        if not firebase_admin._apps:
            firebase_admin.initialize_app(
                credentials.Certificate(FIREBASE_KEY_FILE),
                {'databaseURL': FIREBASE_DB_URL}
            )
        
        cls._initialized = True
        print('✅ Firebase initialized')
        return True
    
    @classmethod
    def get_reference(cls, path: str = '/'):
        """Get Firebase database reference."""
        if not cls._initialized:
            cls.initialize()
        return db.reference(path)
    
    @classmethod
    def http_get(cls, path: str) -> Optional[Any]:
        """HTTP GET from Firebase REST API."""
        url = f"{FIREBASE_URL.rstrip('/')}/{path}.json"
        try:
            resp = cls._http_session.get(url, timeout=REQUEST_TIMEOUT)
            if resp.status_code == 200:
                return resp.json()
            print(f"Firebase GET {path} failed: {resp.status_code}")
        except Exception as e:
            print(f"Firebase GET error: {e}")
        return None
    
    @classmethod
    def http_put(cls, path: str, data: Any) -> bool:
        """HTTP PUT to Firebase REST API."""
        url = f"{FIREBASE_URL.rstrip('/')}/{path}.json"
        try:
            resp = cls._http_session.put(url, json=data, timeout=REQUEST_TIMEOUT)
            return resp.status_code == 200
        except Exception as e:
            print(f"Firebase PUT error: {e}")
        return False
    
    @classmethod
    def get_sensor_data(cls) -> pd.DataFrame:
        """Load all sensor data from Firebase."""
        try:
            data = cls.get_reference('/sensor_data').get()
            if not data:
                return pd.DataFrame()
            
            records = []
            for v in data.values():
                records.append({
                    'timestamp': parse_timestamp(v.get('created_at'), 'firebase'),
                    'temperature': validate_sensor_value(v.get('temperature', 0), 'temperature'),
                    'humidity': validate_sensor_value(v.get('humidity', 0), 'humidity'),
                    'soil': validate_sensor_value(v.get('soil', 0), 'soil')
                })
            
            df = pd.DataFrame(records)
            df = df.dropna(subset=['timestamp'])
            df = df.sort_values('timestamp').reset_index(drop=True)
            return df
        except Exception as e:
            print(f"Error loading sensor data: {e}")
            return pd.DataFrame()
    
    @classmethod
    def save_sensor_reading(cls, data: Dict) -> bool:
        """Save a single sensor reading."""
        try:
            timestamp_key = format_timestamp_key(data['created_at'])
            
            record = {
                'created_at': data['created_at'],
                'temperature': validate_sensor_value(data.get('temperature', 0), 'temperature'),
                'humidity': validate_sensor_value(data.get('humidity', 0), 'humidity'),
                'soil': validate_sensor_value(data.get('soil', 0), 'soil')
            }
            
            cls.get_reference(f'/sensor_data/{timestamp_key}').set(record)
            return True
        except Exception as e:
            print(f"Error saving sensor data: {e}")
            return False
    
    @classmethod
    def get_latest_timestamp(cls) -> Optional[str]:
        """Get the most recent timestamp from Firebase."""
        try:
            ref = cls.get_reference('/sensor_data')
            latest = ref.order_by_child('created_at').limit_to_last(1).get()
            if latest:
                return list(latest.values())[0]['created_at']
        except Exception as e:
            print(f"Error getting latest timestamp: {e}")
        return None


# Initialize on load
FirebaseService.initialize()

# Backward-compatible functions
def load_data_from_firebase():
    """Backward compatible function."""
    return FirebaseService.get_sensor_data()

def firebase_get(path):
    """Backward compatible function."""
    return FirebaseService.http_get(path)

def save_to_firebase(data, path):
    """Backward compatible function."""
    return FirebaseService.http_put(path, data)

print("✅ Firebase Service ready")


## 📊 6. Data Service

In [ ]:
# =============================================================================
# DATA SERVICE - Unified data loading and sync
# =============================================================================

class DataService:
    """Unified data access layer."""
    
    _http_session = requests.Session()
    
    @classmethod
    def fetch_from_server(cls, feed: str = DEFAULT_FEED, limit: int = BATCH_LIMIT, 
                          before_timestamp: str = None) -> Optional[Dict]:
        """Fetch data from external IoT server."""
        params = {"feed": feed, "limit": limit}
        if before_timestamp:
            params["before_created_at"] = before_timestamp
        
        try:
            resp = cls._http_session.get(
                f"{BASE_URL}/history",
                params=params,
                timeout=REQUEST_TIMEOUT * 6  # Longer timeout for history
            )
            return resp.json()
        except Exception as e:
            print(f"Server fetch error: {e}")
            return None
    
    @classmethod
    def load_iot_data(cls, feed: str = DEFAULT_FEED, limit: int = 50) -> Optional[pd.DataFrame]:
        """Load IoT data from server and return as DataFrame."""
        data = cls.fetch_from_server(feed, limit)
        
        if not data or "data" not in data or not data["data"]:
            return None
        
        df = pd.DataFrame(data["data"])
        
        if "created_at" not in df.columns or "value" not in df.columns:
            return None
        
        df["created_at"] = df["created_at"].apply(lambda x: parse_timestamp(x, 'server'))
        df["value"] = pd.to_numeric(df["value"], errors="coerce")
        df = df.dropna(subset=["created_at", "value"])
        df = df.sort_values("created_at")
        
        return df if not df.empty else None
    
    @classmethod
    def load_all_sensors(cls, limit: int = 50) -> Dict[str, Optional[pd.DataFrame]]:
        """Load all sensor types from server."""
        sensors = {}
        for sensor_type in ['temperature', 'humidity', 'soil']:
            sensors[sensor_type] = cls.load_iot_data(sensor_type, limit)
        return sensors
    
    @classmethod
    def sync_from_server(cls) -> Tuple[str, int]:
        """Sync new data from server to Firebase."""
        messages = ["Starting sync..."]
        
        latest = FirebaseService.get_latest_timestamp()
        messages.append(f"Latest in Firebase: {latest}" if latest else "No existing data")
        
        server_data = cls.fetch_from_server()
        
        if not server_data or "data" not in server_data:
            return "\n".join(messages + ["Error fetching from server"]), 0
        
        # Filter new records
        new_records = []
        for sample in server_data["data"]:
            if not latest or sample["created_at"] > latest:
                new_records.append(sample)
        
        if not new_records:
            return "\n".join(messages + ["No new data to sync"]), 0
        
        # Save new records
        saved = 0
        for sample in new_records:
            try:
                vals = json.loads(sample['value'])
                record = {
                    'created_at': sample['created_at'],
                    'temperature': vals.get('temperature', 0),
                    'humidity': vals.get('humidity', 0),
                    'soil': vals.get('soil', 0)
                }
                if FirebaseService.save_sensor_reading(record):
                    saved += 1
            except Exception as e:
                print(f"Error saving record: {e}")
                continue
        
        messages.append(f"Found {len(new_records)} new records")
        messages.append(f"Saved {saved} records!")
        return "\n".join(messages), saved


# Backward-compatible functions
def load_iot_data(feed: str, limit: int) -> Optional[pd.DataFrame]:
    """Backward compatible function."""
    return DataService.load_iot_data(feed, limit)

def sync_new_data_from_server():
    """Backward compatible function."""
    return DataService.sync_from_server()

print("✅ Data Service ready")


## 🌿 7. Plant Analysis Service

In [ ]:
# =============================================================================
# PLANT ANALYSIS SERVICE
# =============================================================================

# Initialize ML classifier
try:
    clf = pipeline("image-classification", model=PLANT_DISEASE_MODEL)
    print(f"✅ Plant disease model loaded: {PLANT_DISEASE_MODEL}")
except Exception as e:
    print(f"⚠️ Could not load plant disease model: {e}")
    clf = None


class PlantAnalysisService:
    """Plant health analysis and disease detection."""
    
    @classmethod
    def analyze_sensor_status(cls, sensors: Dict[str, Optional[pd.DataFrame]]) -> Dict:
        """
        Analyze plant status from sensor data.
        Returns status dict with issues and warnings.
        """
        result = {
            'status': 'unknown',
            'status_emoji': '❓',
            'issues': [],
            'warnings': [],
            'values': {},
            'details': ''
        }
        
        # Check for missing sensors
        missing = [k for k, v in sensors.items() if v is None or v.empty]
        if missing:
            result['status'] = 'partial'
            result['status_emoji'] = '⚠️'
            result['details'] = f"Missing sensors: {', '.join(missing)}"
            return result
        
        # Get latest values
        for sensor_type, df in sensors.items():
            if df is not None and not df.empty:
                value = float(df["value"].iloc[-1])
                result['values'][sensor_type] = value
                
                is_ok, is_warning, message = check_optimal_range(value, sensor_type)
                
                if not is_ok:
                    result['issues'].append(message)
                elif is_warning:
                    result['warnings'].append(message)
        
        # Determine overall status
        if result['issues']:
            result['status'] = 'bad'
            result['status_emoji'] = '🔴'
            result['details'] = ' ; '.join(result['issues'])
        elif result['warnings']:
            result['status'] = 'warning'
            result['status_emoji'] = '🟡'
            result['details'] = ' ; '.join(result['warnings'])
        else:
            result['status'] = 'ok'
            result['status_emoji'] = '🟢'
            result['details'] = 'All sensors within optimal range'
        
        return result
    
    @classmethod
    def analyze_image(cls, image_path: str, temp: float, humidity: float, 
                      soil: float) -> Tuple[str, str, str, str]:
        """
        Analyze plant image for disease detection.
        Returns: (diagnosis, status_html, alerts, recommendations)
        """
        if clf is None:
            return "Model not available", "", "Error: Model not loaded", ""
        
        # Run image classification
        preds = clf(image_path)
        top = preds[0]
        label = top["label"]
        score = top["score"]
        
        # Analyze sensor conditions
        alerts = []
        advice = []
        
        soil_ok, soil_warn, soil_msg = check_optimal_range(soil, 'soil')
        if not soil_ok or soil_warn:
            alerts.append(f"Soil moisture: {soil_msg}")
            if soil < PLANT_OPTIMAL_RANGES['soil']['min']:
                advice.append("Water the plant")
        
        hum_ok, hum_warn, hum_msg = check_optimal_range(humidity, 'humidity')
        if not hum_ok:
            alerts.append(f"Humidity: {hum_msg}")
            if humidity > PLANT_OPTIMAL_RANGES['humidity']['max']:
                advice.append("Improve ventilation to reduce fungal risk")
        
        temp_ok, temp_warn, temp_msg = check_optimal_range(temp, 'temperature')
        if not temp_ok:
            alerts.append(f"Temperature: {temp_msg}")
            if temp > PLANT_OPTIMAL_RANGES['temperature']['max']:
                advice.append("Move plant to shaded area")
        
        # Determine status from image
        is_healthy = "healthy" in label.lower()
        
        status_html = f'''
        <div style="padding:10px;border-radius:10px;
            background:{'#ddffdd' if is_healthy else '#ffdddd'};
            border:1px solid {'#00aa00' if is_healthy else '#ff0000'};
            font-weight:700;">
            {'🟢 Plant status: HEALTHY' if is_healthy else '🔴 Plant status: DISEASE DETECTED'}
        </div>
        '''
        
        diagnosis = f"Detected: {label} ({score:.2%})"
        alerts_text = "\n".join(alerts) if alerts else "No sensor alerts"
        advice_text = "\n".join(advice) if advice else "No immediate actions needed"
        
        return diagnosis, status_html, alerts_text, advice_text


def analyze_plant(image, temp, humidity, soil):
    """Backward compatible function."""
    return PlantAnalysisService.analyze_image(image, temp, humidity, soil)


def plant_dashboard(limit: int):
    """Generate complete plant dashboard."""
    sensors = DataService.load_all_sensors(limit)
    status = PlantAnalysisService.analyze_sensor_status(sensors)
    
    if status['status'] == 'partial':
        return (
            f"{status['status_emoji']} Partial Data",
            status['details'],
            None, None, None, None
        )
    
    # Create plots
    plots = {}
    for sensor_type, df in sensors.items():
        if df is not None and not df.empty:
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=df["created_at"],
                y=df["value"],
                mode="lines+markers",
                name=sensor_type.title(),
                line=dict(color=COLORS.get(sensor_type, '#000'))
            ))
            fig.update_layout(
                title=f"{sensor_type.title()} Over Time",
                xaxis_title="Time",
                yaxis_title=f"{sensor_type.title()} ({SENSOR_VALIDATION_RANGES[sensor_type]['unit']})",
                height=300
            )
            plots[sensor_type] = fig
    
    # Combined normalized plot
    combined_fig = go.Figure()
    for sensor_type, df in sensors.items():
        if df is not None and not df.empty:
            combined_fig.add_trace(go.Scatter(
                x=df["created_at"],
                y=normalize_series(df["value"]),
                mode="lines",
                name=sensor_type.title(),
                line=dict(color=COLORS.get(sensor_type, '#000'))
            ))
    combined_fig.update_layout(
        title="All Sensors (Normalized)",
        xaxis_title="Time",
        yaxis_title="Normalized Value (0-1)",
        height=300
    )
    
    # Build details text
    values = status['values']
    details = (
        f"{status['details']}\n\n"
        f"Latest readings:\n"
        f"  🌡️ Temperature: {values.get('temperature', 'N/A'):.1f}°C\n"
        f"  💧 Humidity: {values.get('humidity', 'N/A'):.1f}%\n"
        f"  🌱 Soil: {values.get('soil', 'N/A'):.1f}%"
    )
    
    return (
        f"{status['status_emoji']} Plant Status: {status['status'].upper()}",
        details,
        plots.get('temperature'),
        plots.get('humidity'),
        plots.get('soil'),
        combined_fig
    )

print("✅ Plant Analysis Service ready")


## 🤖 8. RAG Service (Search & Retrieval)

In [ ]:
# =============================================================================
# RAG SERVICE - Document indexing and retrieval
# =============================================================================

# Initialize NLP tools
stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))

# Global state for RAG
_rag_state = {
    'public_index': None,
    'doc_map': None,
    'doc_text': {},
    'doc_text_cache': {},
    'generator': None
}


class RAGService:
    """Retrieval-Augmented Generation service."""
    
    INDEX_PATH = "indexes/public_index"
    MAP_PATH = "indexes/doc_map"
    TEXT_PATH = "indexes/doc_text"
    
    # -------------------------------------------------------------------------
    # NLP Preprocessing
    # -------------------------------------------------------------------------
    @staticmethod
    def tokenize(text: str) -> List[str]:
        """Tokenize text into words."""
        return re.findall(r"\w+", (text or "").lower())
    
    @staticmethod
    def remove_stopwords(tokens: List[str]) -> List[str]:
        """Remove stop words."""
        return [t for t in tokens if t not in stop_words]
    
    @staticmethod
    def apply_stemming(tokens: List[str]) -> List[str]:
        """Apply Porter stemming."""
        return [stemmer.stem(t) for t in tokens]
    
    @classmethod
    def preprocess(cls, text: str) -> List[str]:
        """Full preprocessing pipeline."""
        tokens = cls.tokenize(text)
        tokens = cls.remove_stopwords(tokens)
        tokens = cls.apply_stemming(tokens)
        return tokens
    
    # -------------------------------------------------------------------------
    # Index Operations
    # -------------------------------------------------------------------------
    @classmethod
    def build_inverted_index(cls, urls: List[str], doc_texts: Dict) -> Tuple[Dict, Dict]:
        """Build inverted index from documents."""
        inverted = defaultdict(set)
        doc_map = {i: url for i, url in enumerate(urls)}
        
        for doc_id in range(len(urls)):
            text = doc_texts.get(doc_id) or doc_texts.get(str(doc_id)) or ""
            tokens = cls.preprocess(text)
            
            for term in set(tokens):
                inverted[term].add(doc_id)
        
        # Convert to sorted lists
        inverted = {term: sorted(list(ids)) for term, ids in inverted.items()}
        return inverted, doc_map
    
    @classmethod
    def load_index(cls, load_text: bool = False) -> bool:
        """Load index from Firebase."""
        try:
            _rag_state['public_index'] = FirebaseService.http_get(cls.INDEX_PATH) or {}
            _rag_state['doc_map'] = FirebaseService.http_get(cls.MAP_PATH) or {}
            
            # Normalize doc_map
            if isinstance(_rag_state['doc_map'], list):
                _rag_state['doc_map'] = {str(i): v for i, v in enumerate(_rag_state['doc_map'])}
            
            if load_text:
                _rag_state['doc_text'] = FirebaseService.http_get(cls.TEXT_PATH) or {}
            
            print(f"Loaded index: {len(_rag_state['public_index'])} terms, {len(_rag_state['doc_map'])} docs")
            return True
        except Exception as e:
            print(f"Error loading index: {e}")
            return False
    
    # -------------------------------------------------------------------------
    # Search Operations
    # -------------------------------------------------------------------------
    @classmethod
    def search(cls, query: str, k: int = 3) -> Tuple[List[str], List[Dict]]:
        """Search for documents matching query."""
        if _rag_state['public_index'] is None:
            cls.load_index()
        
        q_terms = cls.preprocess(query)
        scores = defaultdict(int)
        
        for term in q_terms:
            for doc_id in (_rag_state['public_index'].get(term, []) or []):
                scores[int(doc_id)] += 1
        
        ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:k]
        
        results = []
        for doc_id, score in ranked:
            url = _rag_state['doc_map'].get(str(doc_id))
            results.append({"doc_id": doc_id, "score": score, "url": url})
        
        return q_terms, results
    
    @classmethod
    def get_doc_text(cls, doc_id: int) -> str:
        """Get document text with caching."""
        key = str(int(doc_id))
        
        if key in _rag_state['doc_text_cache']:
            return _rag_state['doc_text_cache'][key] or ""
        
        try:
            txt = FirebaseService.http_get(f"{cls.TEXT_PATH}/{key}")
            if txt is None:
                txt = ""
            if not isinstance(txt, str):
                txt = str(txt)
            _rag_state['doc_text_cache'][key] = txt
            return txt
        except Exception:
            return ""
    
    # -------------------------------------------------------------------------
    # Generation
    # -------------------------------------------------------------------------
    @classmethod
    def init_generator(cls):
        """Initialize text generation model."""
        if _rag_state['generator'] is not None:
            return
        
        try:
            import torch
            device = 0 if torch.cuda.is_available() else -1
            
            try:
                _rag_state['generator'] = pipeline(
                    "text2text-generation",
                    model=RAG_MODEL_PRIMARY,
                    device=device
                )
                print(f"✅ RAG model loaded: {RAG_MODEL_PRIMARY}")
            except Exception:
                _rag_state['generator'] = pipeline(
                    "text2text-generation",
                    model=RAG_MODEL_FALLBACK,
                    device=device
                )
                print(f"✅ RAG model loaded (fallback): {RAG_MODEL_FALLBACK}")
        except Exception as e:
            print(f"⚠️ Could not load RAG model: {e}")
            _rag_state['generator'] = None
    
    @classmethod
    def generate_answer(cls, query: str, k: int = 3) -> Tuple[List[str], List[Dict], str]:
        """Generate answer using RAG."""
        cls.init_generator()
        
        q_terms, results = cls.search(query, k)
        
        if not results:
            return q_terms, results, "No documents matched the query."
        
        # Build context
        context_parts = []
        for r in results[:3]:
            doc_text = cls.get_doc_text(r['doc_id'])
            snippet = doc_text[:RAG_SNIPPET_CHARS] if doc_text else ""
            context_parts.append(f"[Doc {r['doc_id']}]: {snippet}")
        
        context = "\n".join(context_parts)
        
        if _rag_state['generator'] is None:
            return q_terms, results, f"Based on documents: {context[:500]}..."
        
        prompt = (
            "Answer the question using ONLY the provided context.\n"
            "Be concise (1-3 sentences). Cite sources as [Doc X].\n\n"
            f"Question: {query}\n\n"
            f"Context:\n{context}\n\n"
            "Answer:"
        )
        
        try:
            out = _rag_state['generator'](
                prompt,
                max_new_tokens=RAG_MAX_NEW_TOKENS,
                do_sample=False,
                num_beams=1
            )[0]["generated_text"]
            return q_terms, results, out.strip()
        except Exception as e:
            return q_terms, results, f"Generation error: {e}"


# Initialize RAG
RAGService.load_index()
RAGService.init_generator()

# Backward-compatible functions
def preprocess_query(query: str):
    return RAGService.preprocess(query)

def search_top_k(query: str, k: int = 3):
    return RAGService.search(query, k)

def rag_generate_answer(query: str, k: int = 3, snippet_chars: int = 160):
    return RAGService.generate_answer(query, k)

def get_doc_text(doc_id: int) -> str:
    return RAGService.get_doc_text(doc_id)

# Aliases for existing variable names
public_index = _rag_state['public_index']
doc_map = _rag_state['doc_map']
gen = _rag_state['generator']

print("✅ RAG Service ready")


## 📈 9. Visualization Functions

In [ ]:
# =============================================================================
# VISUALIZATION FUNCTIONS
# =============================================================================

class Visualizations:
    """Chart and visualization generators."""
    
    @staticmethod
    def create_kpi_card(label: str, value: float, unit: str, 
                        change: float = 0, trend: str = "up",
                        color: str = None) -> str:
        """Create KPI card HTML."""
        color = color or COLORS['temperature']
        icon = "↑" if trend == "up" else ("↓" if trend == "down" else "→")
        trend_class = f"trend-{trend}"
        
        return f'''
        <div class="kpi-card" style="border-left: 4px solid {color}; 
             background: white; padding: 20px; border-radius: 10px; 
             box-shadow: 0 2px 4px rgba(0,0,0,0.1); text-align: center;">
            <p style="color: #666; font-size: 14px; margin: 0;">{label}</p>
            <p style="font-size: 36px; font-weight: bold; margin: 10px 0;">
                {value:.1f}<span style="font-size: 18px;">{unit}</span>
            </p>
            <p style="color: {'#10b981' if trend == 'up' else '#ef4444'}; font-size: 12px;">
                {icon} {change:.1f} from avg
            </p>
        </div>
        '''
    
    @staticmethod
    def create_time_series(df: pd.DataFrame) -> go.Figure:
        """Create time series plot for all sensors."""
        fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                          subplot_titles=('Temperature', 'Humidity', 'Soil Moisture'),
                          vertical_spacing=0.08)
        
        sensors = [
            ('temperature', '°C', COLORS['temperature']),
            ('humidity', '%', COLORS['humidity']),
            ('soil', '%', COLORS['soil'])
        ]
        
        for i, (col, unit, color) in enumerate(sensors, 1):
            if col in df.columns:
                fig.add_trace(
                    go.Scatter(x=df['timestamp'], y=df[col],
                              mode='lines+markers', name=col.title(),
                              line=dict(color=color)),
                    row=i, col=1
                )
        
        fig.update_layout(height=600, showlegend=True,
                         title_text="Sensor Readings Over Time")
        return fig
    
    @staticmethod
    def create_correlation_matrix(df: pd.DataFrame) -> Tuple[str, go.Figure]:
        """Create correlation matrix."""
        cols = ['temperature', 'humidity', 'soil']
        available = [c for c in cols if c in df.columns]
        
        if len(available) < 2:
            return "Not enough data", None
        
        corr = df[available].corr()
        
        fig = go.Figure(data=go.Heatmap(
            z=corr.values,
            x=available,
            y=available,
            colorscale='RdBu',
            zmid=0,
            text=[[f'{v:.2f}' for v in row] for row in corr.values],
            texttemplate='%{text}',
            textfont={"size": 14}
        ))
        
        fig.update_layout(title="Sensor Correlations", height=400)
        
        explanation = "Shows relationships between sensors. Values near 1 or -1 indicate strong correlation."
        return explanation, fig
    
    @staticmethod
    def create_hourly_patterns(df: pd.DataFrame) -> Tuple[str, go.Figure]:
        """Create hourly aggregation chart."""
        if 'timestamp' not in df.columns:
            return "No timestamp data", None
        
        df_copy = df.copy()
        df_copy['hour'] = df_copy['timestamp'].dt.hour
        
        hourly = df_copy.groupby('hour')[['temperature', 'humidity', 'soil']].mean()
        
        fig = go.Figure()
        for col, color in [('temperature', COLORS['temperature']), 
                          ('humidity', COLORS['humidity']),
                          ('soil', COLORS['soil'])]:
            if col in hourly.columns:
                fig.add_trace(go.Scatter(
                    x=hourly.index, y=hourly[col],
                    mode='lines+markers', name=col.title(),
                    line=dict(color=color)
                ))
        
        fig.update_layout(
            title="Average Values by Hour",
            xaxis_title="Hour (0-23)",
            yaxis_title="Value",
            height=400
        )
        
        return "Shows daily patterns in sensor readings", fig
    
    @staticmethod
    def create_distribution(df: pd.DataFrame) -> Tuple[str, go.Figure]:
        """Create distribution histograms."""
        fig = make_subplots(rows=1, cols=3,
                          subplot_titles=('Temperature', 'Humidity', 'Soil'))
        
        for i, (col, color) in enumerate([
            ('temperature', COLORS['temperature']),
            ('humidity', COLORS['humidity']),
            ('soil', COLORS['soil'])
        ], 1):
            if col in df.columns:
                fig.add_trace(
                    go.Histogram(x=df[col], name=col.title(),
                               marker_color=color, opacity=0.7),
                    row=1, col=i
                )
        
        fig.update_layout(height=350, showlegend=False,
                         title_text="Value Distributions")
        
        return "Distribution of sensor values", fig
    
    @staticmethod
    def create_moving_average(df: pd.DataFrame, variable: str) -> Tuple[str, go.Figure]:
        """Create moving average plot."""
        if variable not in df.columns:
            return f"Variable {variable} not found", None
        
        df_sorted = df.sort_values('timestamp').copy()
        
        fig = go.Figure()
        
        # Raw data
        fig.add_trace(go.Scatter(
            x=df_sorted['timestamp'], y=df_sorted[variable],
            mode='lines', name='Raw', opacity=0.5,
            line=dict(color='gray')
        ))
        
        # Moving averages
        for window, color in [(5, '#3b82f6'), (10, '#ef4444'), (20, '#10b981')]:
            if len(df_sorted) >= window:
                ma = df_sorted[variable].rolling(window=window).mean()
                fig.add_trace(go.Scatter(
                    x=df_sorted['timestamp'], y=ma,
                    mode='lines', name=f'MA-{window}',
                    line=dict(color=color, width=2)
                ))
        
        fig.update_layout(
            title=f"Moving Averages - {variable.title()}",
            height=400
        )
        
        return "Smoothed trends over time", fig


# Backward-compatible functions
def create_kpi_cards(df):
    if df.empty:
        return "<p>No data</p>"
    
    cards = []
    for col, color in [('temperature', COLORS['temperature']),
                       ('humidity', COLORS['humidity']),
                       ('soil', COLORS['soil'])]:
        if col in df.columns:
            current = df[col].iloc[-1]
            avg = df[col].mean()
            unit = SENSOR_VALIDATION_RANGES[col]['unit']
            cards.append(Visualizations.create_kpi_card(
                col.title(), current, unit, current - avg, 
                "up" if current > avg else "down", color
            ))
    
    return f'<div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 20px;">{" ".join(cards)}</div>'


def create_stat_cards_html(df):
    if df.empty:
        return "<p>No data</p>"
    
    stats = []
    for col in ['temperature', 'humidity', 'soil']:
        if col in df.columns:
            stats.append(f'''
            <div style="padding: 15px; background: #f5f5f5; border-radius: 8px;">
                <b>{col.title()}</b><br>
                Min: {df[col].min():.1f} | Max: {df[col].max():.1f} | Avg: {df[col].mean():.1f}
            </div>
            ''')
    
    return f'<div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px;">{" ".join(stats)}</div>'


def create_time_series_plot(df):
    return Visualizations.create_time_series(df)


def calculate_correlations(df):
    return Visualizations.create_correlation_matrix(df)


def hourly_patterns(df):
    return Visualizations.create_hourly_patterns(df)


def daily_patterns(df):
    if df.empty:
        return "No data", None
    df_copy = df.copy()
    df_copy['date'] = df_copy['timestamp'].dt.date
    daily = df_copy.groupby('date')[['temperature', 'humidity', 'soil']].mean()
    
    fig = go.Figure()
    for col, color in [('temperature', COLORS['temperature']),
                       ('humidity', COLORS['humidity']),
                       ('soil', COLORS['soil'])]:
        if col in daily.columns:
            fig.add_trace(go.Scatter(
                x=[str(d) for d in daily.index], y=daily[col],
                mode='lines+markers', name=col.title(),
                line=dict(color=color)
            ))
    fig.update_layout(title="Daily Averages", height=400)
    return "Daily average values", fig


def distribution_analysis(df):
    return Visualizations.create_distribution(df)


def time_series_decomposition(df, variable):
    return Visualizations.create_moving_average(df, variable)


print("✅ Visualization functions loaded")


## 📄 10. Report Generator

In [ ]:
# =============================================================================
# REPORT GENERATOR
# =============================================================================

class ReportGenerator:
    """Generate DOCX reports from sensor data."""
    
    @staticmethod
    def create_report(df: pd.DataFrame) -> str:
        """Create a DOCX report and return the file path."""
        if df is None or df.empty:
            raise ValueError("No data available for report")
        
        doc = Document()
        
        # Title
        title = doc.add_heading("CloudGarden Plant Health Report", level=0)
        title.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # Date
        date_para = doc.add_paragraph()
        date_para.add_run(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
        date_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        doc.add_paragraph()
        
        # Executive Summary
        doc.add_heading("Executive Summary", level=1)
        
        # Analyze data
        issues = []
        for col in ['temperature', 'humidity', 'soil']:
            if col in df.columns:
                avg = df[col].mean()
                ranges = PLANT_OPTIMAL_RANGES.get(col, {})
                if avg < ranges.get('min', 0):
                    issues.append(f"{col.title()} is low (avg: {avg:.1f})")
                elif avg > ranges.get('max', 100):
                    issues.append(f"{col.title()} is high (avg: {avg:.1f})")
        
        if issues:
            summary = "Issues detected:\n" + "\n".join(f"- {i}" for i in issues)
        else:
            summary = "All environmental conditions are within optimal ranges."
        
        doc.add_paragraph(summary)
        
        # Environmental Conditions Table
        doc.add_heading("Environmental Conditions", level=1)
        
        table = doc.add_table(rows=1, cols=5)
        table.style = 'Table Grid'
        headers = table.rows[0].cells
        headers[0].text = "Sensor"
        headers[1].text = "Current"
        headers[2].text = "Average"
        headers[3].text = "Min"
        headers[4].text = "Max"
        
        for col in ['temperature', 'humidity', 'soil']:
            if col in df.columns:
                row = table.add_row().cells
                row[0].text = col.title()
                row[1].text = f"{df[col].iloc[-1]:.1f}"
                row[2].text = f"{df[col].mean():.1f}"
                row[3].text = f"{df[col].min():.1f}"
                row[4].text = f"{df[col].max():.1f}"
        
        # Statistics
        doc.add_heading("Statistical Summary", level=1)
        
        stats_text = f"""
Data Points: {len(df)}
Time Range: {df['timestamp'].min()} to {df['timestamp'].max()}

Temperature: Mean={df['temperature'].mean():.1f}°C, Std={df['temperature'].std():.1f}°C
Humidity: Mean={df['humidity'].mean():.1f}%, Std={df['humidity'].std():.1f}%
Soil Moisture: Mean={df['soil'].mean():.1f}%, Std={df['soil'].std():.1f}%
"""
        doc.add_paragraph(stats_text)
        
        # Save to temp file
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.docx')
        doc.save(temp_file.name)
        
        return temp_file.name


def generate_report_screen(samples: int):
    """Generate report from sensor data."""
    try:
        # Load data
        sensors = DataService.load_all_sensors(samples)
        
        # Combine sensor data
        dfs = []
        for sensor_type, df in sensors.items():
            if df is not None and not df.empty:
                df_renamed = df.rename(columns={'value': sensor_type, 'created_at': 'timestamp'})
                dfs.append(df_renamed[['timestamp', sensor_type]])
        
        if not dfs:
            return "No data available", None
        
        # Merge all sensors
        combined = dfs[0]
        for df in dfs[1:]:
            combined = combined.merge(df, on='timestamp', how='outer')
        combined = combined.sort_values('timestamp').reset_index(drop=True)
        
        # Generate report
        file_path = ReportGenerator.create_report(combined)
        
        return f"✅ Report generated with {len(combined)} data points", file_path
    
    except Exception as e:
        return f"❌ Error: {str(e)}", None


print("✅ Report Generator ready")


## 🎨 11. CSS Styles

In [ ]:
# =============================================================================
# CSS STYLES
# =============================================================================

CUSTOM_CSS = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap');
* { font-family: 'Inter', sans-serif; }

.kpi-card {
    background: white;
    padding: 24px;
    border-radius: 12px;
    box-shadow: 0 1px 3px rgba(0,0,0,0.12);
    text-align: center;
    border-left: 4px solid;
}

.kpi-label { color: #6b7280; font-size: 14px; font-weight: 600; }
.kpi-value { font-size: 48px; font-weight: 700; color: #1f2937; }
.trend-up { color: #10b981; }
.trend-down { color: #ef4444; }

.status-badge {
    display: inline-flex;
    align-items: center;
    padding: 4px 12px;
    border-radius: 16px;
    font-size: 12px;
    font-weight: 600;
}

.explanation-card {
    padding: 16px;
    border-radius: 8px;
    margin: 10px 0;
}

.legend-card {
    padding: 14px;
    border: 1px solid var(--border-color-primary);
    border-radius: 10px;
    margin-top: 14px;
}
"""

print("✅ CSS loaded")


## 🖥️ 12. UI Components (Gradio Tabs)

In [ ]:
# =============================================================================
# UI COMPONENTS - TAB BUILDERS
# =============================================================================

def build_realtime_dashboard_tab():
    """Build the realtime plant dashboard tab."""
    gr.Markdown("<h3>🌿 Overall Plant Status (Real-Time)</h3>")
    
    samples = gr.Slider(1, 200, value=20, step=1, label="Number of Samples")
    overall_btn = gr.Button("Update Plant Dashboard", variant="primary")
    
    overall_status = gr.Textbox(label="Overall Status", lines=1)
    overall_info = gr.Textbox(label="Status Details", lines=4)
    
    with gr.Row():
        gr.Markdown(f"""
        <div class="legend-card">
            <h4>🌿 Plant Status</h4>
            <span style="color:{STATUS_OK_COLOR};font-size:26px;">●</span> <b>Healthy</b> - All values normal<br>
            <span style="color:{STATUS_WARN_COLOR};font-size:26px;">●</span> <b>Warning</b> - Near threshold<br>
            <span style="color:{STATUS_BAD_COLOR};font-size:26px;">●</span> <b>Not OK</b> - Out of range
        </div>
        """)
        
        gr.Markdown(f"""
        <div class="legend-card">
            <h4>ℹ️ Valid Ranges</h4>
            <span style="color:{COLOR_TEMP};font-size:26px;">●</span> 🌡️ Temperature: 18–32°C<br>
            <span style="color:{COLOR_HUM};font-size:26px;">●</span> 💧 Humidity: 35–75%<br>
            <span style="color:{COLOR_SOIL};font-size:26px;">●</span> 🌱 Soil: 20–60%
        </div>
        """)
    
    gr.Markdown("<h2 style='text-align:center;'>📈 Plant Sensor Graphs</h2>")
    
    with gr.Row():
        plot_temp = gr.Plot(label="Temperature")
        plot_hum = gr.Plot(label="Humidity")
    
    with gr.Row():
        plot_soil = gr.Plot(label="Soil Moisture")
        plot_combined = gr.Plot(label="Combined (Normalized)")
    
    overall_btn.click(
        fn=plant_dashboard,
        inputs=[samples],
        outputs=[overall_status, overall_info, plot_temp, plot_hum, plot_soil, plot_combined]
    )


def build_iot_dashboard_tab():
    """Build the IoT analytics dashboard tab."""
    gr.Markdown("### 📈 Comprehensive Sensor Analytics")
    
    refresh_btn = gr.Button("🔄 Refresh All Data", variant="primary")
    
    # KPI Cards
    gr.Markdown("#### 📌 Current Readings")
    kpi_html = gr.HTML()
    
    # Statistics
    gr.Markdown("#### 📊 Statistical Summary")
    stats_html = gr.HTML()
    
    # Time Series
    gr.Markdown("#### 📈 Time Series")
    ts_plot = gr.Plot()
    
    # Correlation
    gr.Markdown("#### 🔗 Correlation Analysis")
    corr_card = gr.HTML()
    corr_plot = gr.Plot()
    
    # Hourly Patterns
    gr.Markdown("#### ⏰ Hourly Patterns")
    hourly_card = gr.HTML()
    hourly_plot = gr.Plot()
    
    # Daily Trends
    gr.Markdown("#### 📅 Daily Trends")
    daily_card = gr.HTML()
    daily_plot = gr.Plot()
    
    # Distribution
    gr.Markdown("#### 📊 Distributions")
    dist_card = gr.HTML()
    dist_plot = gr.Plot()
    
    # Moving Averages
    gr.Markdown("#### 📉 Moving Averages")
    with gr.Row():
        ma_variable = gr.Dropdown(
            choices=['temperature', 'humidity', 'soil'],
            value='temperature',
            label='Select Variable'
        )
        ma_btn = gr.Button("Generate")
    ma_card = gr.HTML()
    ma_plot = gr.Plot()
    
    def dashboard_screen():
        df = load_data_from_firebase()
        if df.empty:
            empty = "<div style='padding:20px;text-align:center;'>No data. Click Sync Data!</div>"
            return empty, empty, None, empty, None, empty, None, empty, None, empty, None
        
        kpi = create_kpi_cards(df)
        stats = create_stat_cards_html(df)
        ts = create_time_series_plot(df)
        corr_c, corr_p = calculate_correlations(df)
        hour_c, hour_p = hourly_patterns(df)
        day_c, day_p = daily_patterns(df)
        dist_c, dist_p = distribution_analysis(df)
        
        return kpi, stats, ts, corr_c, corr_p, hour_c, hour_p, day_c, day_p, dist_c, dist_p
    
    def dashboard_ma(variable):
        df = load_data_from_firebase()
        if df.empty:
            return "No data", None
        return time_series_decomposition(df, variable)
    
    refresh_btn.click(
        dashboard_screen,
        outputs=[kpi_html, stats_html, ts_plot, corr_card, corr_plot,
                hourly_card, hourly_plot, daily_card, daily_plot, dist_card, dist_plot]
    )
    
    ma_btn.click(dashboard_ma, inputs=[ma_variable], outputs=[ma_card, ma_plot])


def build_generate_report_tab():
    """Build the report generation tab."""
    gr.Markdown("## 📄 Generate Report")
    gr.Markdown("Generate a Word (DOCX) report based on sensor data.")
    
    report_samples = gr.Slider(5, 200, value=20, step=1, label="Samples per sensor")
    report_btn = gr.Button("📥 Generate & Download Report", variant="primary")
    report_status = gr.Textbox(label="Status", lines=2)
    report_file = gr.File(label="Download DOCX")
    
    report_btn.click(
        fn=generate_report_screen,
        inputs=[report_samples],
        outputs=[report_status, report_file]
    )


def build_plant_disease_detection_tab():
    """Build the plant disease detection tab."""
    gr.Markdown("## 🖼️ Plant Disease Detection")
    
    with gr.Row():
        with gr.Column(scale=2):
            image = gr.Image(type="filepath", label="Upload plant image", sources=["upload"])
            temp = gr.Slider(0, 45, value=25, label="Temperature (°C)")
            humidity = gr.Slider(0, 100, value=50, label="Humidity (%)")
            soil = gr.Slider(0, 100, value=50, label="Soil Moisture (%)")
            run_btn = gr.Button("Analyze Plant", variant="primary")
        
        with gr.Column(scale=2):
            diagnosis = gr.Textbox(label="Diagnosis")
            status = gr.HTML(label="Status")
            alerts = gr.Textbox(label="Alerts", lines=5)
            recommendations = gr.Textbox(label="Recommendations", lines=5)
    
    run_btn.click(
        fn=analyze_plant,
        inputs=[image, temp, humidity, soil],
        outputs=[diagnosis, status, alerts, recommendations]
    )


def build_rag_chat_tab():
    """Build the RAG chat tab."""
    gr.Markdown("## 💬 RAG Chat")
    gr.Markdown("Ask questions about plant diseases using document retrieval.")
    
    question = gr.Textbox(label="🔍 Query", placeholder="e.g., What causes leaf spots?", lines=2)
    n_results = gr.Slider(1, 5, value=3, step=1, label="Top-K documents")
    run_btn = gr.Button("Run", variant="primary")
    
    retrieved_df = gr.Dataframe(headers=["doc_id", "score", "url"], label="📄 Retrieved Documents")
    answer_box = gr.Textbox(label="🤖 RAG Answer", lines=10)
    
    def query_rag(q, k):
        if not q.strip():
            return [], "Please enter a question."
        
        q_terms, results, answer = RAGService.generate_answer(q, k=int(k))
        rows = [[r.get("doc_id"), r.get("score"), r.get("url")] for r in results]
        return rows, answer
    
    run_btn.click(query_rag, inputs=[question, n_results], outputs=[retrieved_df, answer_box])


def build_sync_data_tab():
    """Build the data sync tab."""
    gr.Markdown("## 🔄 Sync Data")
    gr.Markdown("Upload IoT Data to Firebase")
    
    sync_btn = gr.Button("🔄 Sync from Server", variant="primary")
    sync_status = gr.Textbox(label="Sync Status", lines=5)
    
    def sync_action():
        msg, count = DataService.sync_from_server()
        return msg
    
    sync_btn.click(sync_action, outputs=[sync_status])


print("✅ UI Components loaded")


## 🚀 13. App Builder

In [ ]:
# =============================================================================
# TAB REGISTRY & APP BUILDER
# =============================================================================

# Tab configuration - single place to add/remove tabs
TABS = [
    ("🌱 Realtime Dashboard", build_realtime_dashboard_tab),
    ("📊 IoT Dashboard", build_iot_dashboard_tab),
    ("📄 Generate Report", build_generate_report_tab),
    ("🖼️ Plant Disease Detection", build_plant_disease_detection_tab),
    ("💬 RAG Chat", build_rag_chat_tab),
    ("🔄 Sync Data", build_sync_data_tab),
]


def build_app() -> gr.Blocks:
    """Build the complete Gradio application."""
    with gr.Blocks(title=APP_TITLE, css=CUSTOM_CSS) as demo:
        # Header
        gr.Markdown(f"<h1 style='text-align:left;font-size:30px;font-weight:700;'>{APP_TITLE}</h1>")
        gr.Markdown(f"<p style='text-align:left;font-size:18px;color:#666;'>{APP_SUBTITLE}</p>")
        
        # Build all tabs
        with gr.Tabs():
            for tab_name, tab_builder in TABS:
                with gr.Tab(tab_name):
                    tab_builder()
    
    return demo


print("✅ App Builder ready")
print(f"   Configured tabs: {len(TABS)}")
for name, _ in TABS:
    print(f"   - {name}")


## 🎬 14. Launch Application

In [ ]:
# =============================================================================
# LAUNCH APPLICATION
# =============================================================================

if __name__ == "__main__":
    print("\n" + "="*50)
    print("🌱 Starting CloudGarden Application...")
    print("="*50 + "\n")
    
    app = build_app()
    app.launch(share=True, debug=True)
